In [ ]:
!nvidia-smi --query-gpu=name,utilization.gpu,utilization.memory,memory.total,memory.free --format=csv

name, utilization.gpu [%], utilization.memory [%], memory.total [MiB], memory.free [MiB]
NVIDIA A100-SXM4-40GB, 0 %, 0 %, 40960 MiB, 40511 MiB


In [ ]:
import torch
device = torch.device("cuda")

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!pip install --upgrade pip setuptools wheel
!apt-get update
!apt-get install -y build-essential libopenblas-dev libomp-dev
!apt-get install -y cuda-libraries-12-2

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
!pip install torch --pre --index-url https://download.pytorch.org/whl/nightly/cu122

Looking in indexes: https://download.pytorch.org/whl/nightly/cu122


In [ ]:
!sudo ln -sf /usr/local/cuda-12.2/lib64/libcudart.so.12 /usr/lib/x86_64-linux-gnu/libcudart.so.11
!sudo ln -sf /usr/local/cuda-12.2/lib64/libcudart.so.12 /usr/lib/x86_64-linux-gnu/libcudart.so.11.0

In [ ]:
!ls -l /usr/lib/x86_64-linux-gnu/libcudart.so.11*

lrwxrwxrwx 1 root root 42 Dec 15 03:49 /usr/lib/x86_64-linux-gnu/libcudart.so.11 -> /usr/local/cuda-12.2/lib64/libcudart.so.12
lrwxrwxrwx 1 root root 42 Dec 15 03:49 /usr/lib/x86_64-linux-gnu/libcudart.so.11.0 -> /usr/local/cuda-12.2/lib64/libcudart.so.12


In [ ]:
!export FORCE_CUDA="1"
!export TORCH_CUDA_ARCH_LIST="7.5"
!export PATH="/usr/local/cuda-12.2/bin:$PATH"
!export LD_LIBRARY_PATH="/usr/local/cuda-12.2/lib64:$LD_LIBRARY_PATH"

In [ ]:
!pip uninstall -y flash-attn

Found existing installation: flash-attn 2.7.2.post1
Uninstalling flash-attn-2.7.2.post1:
  Successfully uninstalled flash-attn-2.7.2.post1


In [ ]:
!git clone https://github.com/HazyResearch/flash-attention.git
%cd flash-attention

fatal: destination path 'flash-attention' already exists and is not an empty directory.
/content/flash-attention


In [ ]:
!git checkout v2.7.2.post1
!git submodule update --init --recursive

HEAD is now at f86e3dd [CI] Use MAX_JOBS=1 with nvcc 12.3, don't need OLD_GENERATOR_PATH


In [ ]:
!pip install . --no-build-isolation

Processing /content/flash-attention
  Preparing metadata (setup.py) ... done
  Created wheel for flash_attn: filename=flash_attn-2.7.2.post1-cp310-cp310-linux_x86_64.whl size=190160474 sha256=0b454d9e650bfc437cc71335080172a5d05f51eab355636c9d5b7321fec7318e
  Stored in directory: /root/.cache/pip/wheels/98/cf/3b/d132219792be47c1a416734b31d5be638f6a6e282470b490c6
Successfully built flash_attn


In [ ]:
import flash_attn
print("FlashAttention 2.4 installed successfully!")

FlashAttention 2.4 installed successfully!


In [ ]:
!pip show flash-attn

Name: flash-attn
Version: 2.7.2.post1
Summary: Flash Attention: Fast and Memory-Efficient Exact Attention
Home-page: https://github.com/Dao-AILab/flash-attention
Author: Tri Dao
Author-email: tri@tridao.me
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: einops, torch
Required-by: 


In [ ]:
!nvidia-smi

Sun Dec 15 03:50:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              42W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install transformers rouge rouge-score nltk datasets

In [ ]:
import torch
from torch.amp import autocast
import math
import time
from transformers import Trainer, TrainingArguments, GPT2LMHeadModel, GPT2TokenizerFast
from transformers.models.gpt2.modeling_gpt2 import GPT2Attention, GPT2Config
from flash_attn.flash_attn_interface import flash_attn_func
from datasets import load_dataset
from tabulate import tabulate
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import wandb

In [ ]:
# Configuration
seq_len = 512 # Maximum sequence length for both input and labels
train_subset_size = 1000  # Training size
val_subset_size = 200     # Validation size

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Load CNN/DailyMail dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Subset the dataset
train_dataset = dataset["train"].shuffle(seed=42).select(range(train_subset_size))  # Take 1,000 examples
val_dataset = dataset["validation"].shuffle(seed=42).select(range(val_subset_size))  # Take 200 examples

# Tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Set padding token to be EOS token

# Preprocess dataset
def preprocess_function(example):
    input_text = example["article"]
    target_text = example["highlights"]

    # Tokenize the input text with the updated sequence length
    model_inputs = tokenizer(
        input_text, max_length=seq_len, truncation=True, padding="max_length"
    )

    # Tokenize the target text
    labels = tokenizer(
        target_text, max_length=seq_len, truncation=True, padding="max_length"
    )

    # Mask padding tokens in labels to -100 so they are ignored by the loss function
    labels["input_ids"] = [
        (label if label != tokenizer.pad_token_id else -100) for label in labels["input_ids"]
    ]

    # Add labels to the model inputs
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing to the subsets
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)

# Verify dataset columns
print(f"Train Dataset Columns: {tokenized_train_dataset.column_names}")
print(f"Validation Dataset Columns: {tokenized_val_dataset.column_names}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Train Dataset Columns: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels']
Validation Dataset Columns: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels']


In [ ]:
# ALiBi Score Function
def get_alibi_slopes(nheads):
    def get_slopes_power_of_2(n):
        start = 2.0 ** (-2.0 ** -(math.log2(n) - 3))
        return [start * (2.0 ** (i / 2.0)) for i in range(n)]

    if math.log2(nheads).is_integer():
        slopes = get_slopes_power_of_2(nheads)
    else:
        closest_power_of_2 = 2 ** math.floor(math.log2(nheads))
        slopes = get_slopes_power_of_2(closest_power_of_2)
        slopes += [slopes[-1] * 2.0 ** ((i + 1) / 2.0) for i in range(nheads - closest_power_of_2)]
    return torch.tensor(slopes, dtype=torch.float32, device=device)

In [ ]:
# Helper Functions
def benchmark_model(model, tokenizer, texts):
    """Benchmark model for inference time and memory usage."""
    inputs = tokenizer(texts, return_tensors="pt", max_length=seq_len, padding=True, truncation=True).to(device)
    torch.cuda.reset_peak_memory_stats()
    times = []

    # Warmup
    for _ in range(5):
        with torch.no_grad(), autocast(device_type='cuda'):  # Use device_type='cuda'
            _ = model(**inputs)

    # Measure time
    with torch.no_grad(), autocast(device_type='cuda'):  # Use device_type='cuda'
        for _ in range(10):
            start = time.time()
            _ = model(**inputs)
            end = time.time()
            times.append(end - start)

    avg_time = sum(times) / len(times)
    max_mem = torch.cuda.max_memory_allocated() / (1024**2)
    return avg_time * 1000, max_mem



def generate_text(model, tokenizer, texts, max_new_tokens=100, num_beams=5):
    """Generate text using the model."""
    outputs = []
    for text in texts:
        inputs = tokenizer(
            text, return_tensors="pt", max_length=seq_len, truncation=True, padding=True
        ).to(device)
        attention_mask = inputs["attention_mask"]

        model.config.pad_token_id = tokenizer.eos_token_id

        with torch.no_grad(), autocast(device_type='cuda'):  # Use device_type='cuda'
            output_ids = model.generate(
                inputs["input_ids"],
                attention_mask=attention_mask,
                max_new_tokens=max_new_tokens,
                num_beams=num_beams,
                no_repeat_ngram_size=2,
                early_stopping=True,
            )
        outputs.append(tokenizer.decode(output_ids[0], skip_special_tokens=True))
    return outputs

def compute_metrics(predicted, references):
    """Compute ROUGE and BLEU metrics."""
    rouge = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    rouge_scores = {"rouge1": 0, "rouge2": 0, "rougeL": 0}
    bleu_score = 0

    for pred, ref in zip(predicted, references):
        rouge_res = rouge.score(pred, ref)
        rouge_scores["rouge1"] += rouge_res["rouge1"].fmeasure
        rouge_scores["rouge2"] += rouge_res["rouge2"].fmeasure
        rouge_scores["rougeL"] += rouge_res["rougeL"].fmeasure

        smoothing_fn = SmoothingFunction().method4
        bleu_score += sentence_bleu([ref.split()], pred.split(), smoothing_function=smoothing_fn)

    for key in rouge_scores:
        rouge_scores[key] /= len(predicted)
    bleu_score /= len(predicted)
    return rouge_scores, bleu_score

In [ ]:
class FlashGPT2Attention(GPT2Attention):
    def forward(
        self,
        hidden_states,
        layer_past=None,
        attention_mask=None,
        head_mask=None,
        use_cache=False,
        output_attentions=False,
    ):
        bsz, slen, hidden_size = hidden_states.size()
        assert hidden_size == self.embed_dim

        # Project hidden states to query, key, and value
        qkv_proj = self.c_attn(hidden_states)
        query, key, value = qkv_proj.split(self.embed_dim, dim=2)

        # Ensure tensors are in fp16
        query = query.view(bsz, slen, self.num_heads, self.head_dim).to(torch.float16)
        key = key.view(bsz, slen, self.num_heads, self.head_dim).to(torch.float16)
        value = value.view(bsz, slen, self.num_heads, self.head_dim).to(torch.float16)

        if layer_past is not None:
            past_key, past_value = layer_past
            key = torch.cat((past_key, key), dim=1)
            value = torch.cat((past_value, value), dim=1)
        present = (key, value) if use_cache else None

        scale = 1.0 / math.sqrt(self.head_dim)

        # Call FlashAttention
        attn_output = flash_attn_func(
            query,
            key,
            value,
            causal=True,
            softmax_scale=scale,
        )

        # Reshape output
        attn_output = attn_output.reshape(bsz, slen, hidden_size)
        attn_output = self.c_proj(attn_output)
        attn_output = self.resid_dropout(attn_output)

        return attn_output, present

In [ ]:
class FlashGPT2AttentionWithALiBi(GPT2Attention):
    def __init__(self, config, is_cross_attention=False, layer_idx=None):
        """
        Flash Attention with ALiBi (Attention with Linear Biases).

        Args:
            config: GPT2 configuration object.
            is_cross_attention: Whether this layer is for cross-attention.
            layer_idx: Layer index (if needed for custom configurations).
        """
        super().__init__(config, is_cross_attention=is_cross_attention, layer_idx=layer_idx)

    def forward(
        self,
        hidden_states,
        layer_past=None,
        attention_mask=None,
        head_mask=None,
        use_cache=False,
        output_attentions=False,
    ):
        bsz, slen, hidden_size = hidden_states.size()
        assert hidden_size == self.embed_dim, "Hidden size must match embedding dimension."

        # Project hidden states to query, key, and value
        qkv_proj = self.c_attn(hidden_states)
        query, key, value = qkv_proj.split(self.embed_dim, dim=2)

        query = query.view(bsz, slen, self.num_heads, self.head_dim).transpose(1, 2)  # [bsz, num_heads, slen, head_dim]
        key = key.view(bsz, slen, self.num_heads, self.head_dim).transpose(1, 2)      # [bsz, num_heads, slen, head_dim]
        value = value.view(bsz, slen, self.num_heads, self.head_dim).transpose(1, 2)  # [bsz, num_heads, slen, head_dim]

        if layer_past is not None:
            past_key, past_value = layer_past
            key = torch.cat((past_key, key), dim=2)  # Concatenate along sequence length
            value = torch.cat((past_value, value), dim=2)
        present = (key, value) if use_cache else None

        scale = 1.0 / math.sqrt(self.head_dim)

        # Generate ALiBi slopes and bias
        alibi_slopes = get_alibi_slopes(self.num_heads, device=hidden_states.device).to(hidden_states.dtype)
        alibi = alibi_slopes.view(1, self.num_heads, 1, 1)  # Shape: [1, num_heads, 1, 1]

        # Generate position indices
        position_ids = torch.arange(slen, device=hidden_states.device)  # Shape: [slen]
        relative_positions = position_ids.unsqueeze(0) - position_ids.unsqueeze(-1)  # Shape: [slen, slen]
        relative_positions = relative_positions.to(hidden_states.dtype)

        # Generate ALiBi bias
        alibi_bias = alibi * relative_positions.unsqueeze(0)  # Final shape: [1, num_heads, slen, slen]

        # Compute attention scores
        attn_scores = torch.einsum("bnqd,bnkd->bnqk", query, key) * scale  # Correct einsum

        # Add ALiBi bias
        attn_scores = attn_scores + alibi_bias  # Shape: [bsz, num_heads, slen, slen]

        if attention_mask is not None:
            assert attention_mask.dim() == 4, "Attention mask must be 4D: [bsz, 1, 1, slen]"
            attn_scores += attention_mask

        # Compute attention probabilities
        attn_probs = torch.softmax(attn_scores, dim=-1)
        attn_probs = self.attn_dropout(attn_probs)  # Apply dropout

        # Compute attention output
        attn_output = torch.einsum("bnqk,bnvd->bnqd", attn_probs, value)  # Shape: [bsz, num_heads, slen, head_dim]
        attn_output = attn_output.transpose(1, 2).contiguous().view(bsz, slen, hidden_size)  # [bsz, slen, hidden_size]
        attn_output = self.c_proj(attn_output)
        attn_output = self.resid_dropout(attn_output)

        return (attn_output, present) if use_cache else (attn_output, None)

In [ ]:
# Load Models

# Load the standard model with default GPT-2 attention
model_standard = GPT2LMHeadModel.from_pretrained("gpt2").to(device).eval()

# Load the FlashAttention-enabled model
model_flash = GPT2LMHeadModel.from_pretrained("gpt2").to(device).eval()
for block in model_flash.transformer.h:
    block.attn = FlashGPT2Attention(
        config=model_flash.config,
        is_cross_attention=block.attn.is_cross_attention,
        layer_idx=block.attn.layer_idx
    ).to(device)

# Load the FlashAttention + ALiBi-enabled model
model_flash_with_alibi = GPT2LMHeadModel.from_pretrained("gpt2").to(device).eval()
for block in model_flash_with_alibi.transformer.h:
    block.attn = FlashGPT2AttentionWithALiBi(
        config=model_flash_with_alibi.config,
        is_cross_attention=block.attn.is_cross_attention,
        layer_idx=block.attn.layer_idx
    ).to(device)

In [ ]:
# Input texts and reference summaries from validation dataset
input_texts = [example["article"] for example in dataset["validation"].select(range(5))]
reference_summaries = [example["highlights"] for example in dataset["validation"].select(range(5))]

In [ ]:
# Benchmarking
print("Running Benchmarks...")
standard_time, standard_mem = benchmark_model(model_standard, tokenizer, input_texts)
flash_time, flash_mem = benchmark_model(model_flash, tokenizer, input_texts)
flash_alibi_time, flash_alibi_mem = benchmark_model(model_flash_with_alibi, tokenizer, input_texts)

# Generate Texts
standard_outputs = generate_text(model_standard, tokenizer, input_texts)
flash_outputs = generate_text(model_flash, tokenizer, input_texts)
flash_alibi_outputs = generate_text(model_flash_with_alibi, tokenizer, input_texts)

# Compute Metrics
standard_metrics = compute_metrics(standard_outputs, reference_summaries)
flash_metrics = compute_metrics(flash_outputs, reference_summaries)
flash_alibi_metrics = compute_metrics(flash_alibi_outputs, reference_summaries)

# Organize Results
results_efficiency = [
    ["Model", "Inference Time (ms)", "Memory (MB)"],
    ["Standard GPT-2", f"{standard_time:.2f}", f"{standard_mem:.2f}"],
    ["Flash GPT-2", f"{flash_time:.2f}", f"{flash_mem:.2f}"],
    ["Flash + ALiBi GPT-2", f"{flash_alibi_time:.2f}", f"{flash_alibi_mem:.2f}"]
]

results_quality = [
    ["Model", "ROUGE-1", "ROUGE-2", "ROUGE-L", "BLEU"],
    ["Standard GPT-2", f"{standard_metrics[0]['rouge1']:.4f}", f"{standard_metrics[0]['rouge2']:.4f}", f"{standard_metrics[0]['rougeL']:.4f}", f"{standard_metrics[1]:.4f}"],
    ["Flash GPT-2", f"{flash_metrics[0]['rouge1']:.4f}", f"{flash_metrics[0]['rouge2']:.4f}", f"{flash_metrics[0]['rougeL']:.4f}", f"{flash_metrics[1]:.4f}"],
    ["Flash + ALiBi GPT-2", f"{flash_alibi_metrics[0]['rouge1']:.4f}", f"{flash_alibi_metrics[0]['rouge2']:.4f}", f"{flash_alibi_metrics[0]['rougeL']:.4f}", f"{flash_alibi_metrics[1]:.4f}"]
]

print("\n===== Efficiency Comparison =====\n")
print(tabulate(results_efficiency, headers="firstrow", tablefmt="pretty"))

print("\n===== Quality Comparison =====\n")
print(tabulate(results_quality, headers="firstrow", tablefmt="pretty"))

Running Benchmarks...


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Settin


===== Efficiency Comparison =====

+---------------------+---------------------+-------------+
|        Model        | Inference Time (ms) | Memory (MB) |
+---------------------+---------------------+-------------+
|   Standard GPT-2    |        79.61        |   4407.28   |
|     Flash GPT-2     |        17.21        |   3747.49   |
| Flash + ALiBi GPT-2 |        30.94        |   3747.49   |
+---------------------+---------------------+-------------+

===== Quality Comparison =====

+---------------------+---------+---------+---------+--------+
|        Model        | ROUGE-1 | ROUGE-2 | ROUGE-L |  BLEU  |
+---------------------+---------+---------+---------+--------+
|   Standard GPT-2    | 0.1609  | 0.0861  | 0.1177  | 0.0293 |
|     Flash GPT-2     | 0.1618  | 0.0866  | 0.1184  | 0.0296 |
| Flash + ALiBi GPT-2 | 0.1644  | 0.0887  | 0.1202  | 0.0327 |
+---------------------+---------+---------+---------+--------+


In [ ]:
# Initialize wandb
wandb.init(project="intro2llm", name="benchmark_results")

# Log efficiency results
efficiency_table = wandb.Table(columns=["Model", "Inference Time (ms)", "Memory (MB)"])
efficiency_table.add_data("Standard GPT-2", standard_time, standard_mem)
efficiency_table.add_data("Flash GPT-2", flash_time, flash_mem)
efficiency_table.add_data("Flash + ALiBi GPT-2", flash_alibi_time, flash_alibi_mem)
wandb.log({"Efficiency Metrics": efficiency_table})

# Log quality results
quality_table = wandb.Table(columns=["Model", "ROUGE-1", "ROUGE-2", "ROUGE-L", "BLEU"])
quality_table.add_data("Standard GPT-2",
                       standard_metrics[0]['rouge1'],
                       standard_metrics[0]['rouge2'],
                       standard_metrics[0]['rougeL'],
                       standard_metrics[1])
quality_table.add_data("Flash GPT-2",
                       flash_metrics[0]['rouge1'],
                       flash_metrics[0]['rouge2'],
                       flash_metrics[0]['rougeL'],
                       flash_metrics[1])
quality_table.add_data("Flash + ALiBi GPT-2",
                       flash_alibi_metrics[0]['rouge1'],
                       flash_alibi_metrics[0]['rouge2'],
                       flash_alibi_metrics[0]['rougeL'],
                       flash_alibi_metrics[1])
wandb.log({"Quality Metrics": quality_table})

# Log as summary
wandb.summary["Standard Inference Time (ms)"] = standard_time
wandb.summary["Flash Inference Time (ms)"] = flash_time
wandb.summary["Flash + ALiBi Inference Time (ms)"] = flash_alibi_time

wandb.summary["Standard Memory (MB)"] = standard_mem
wandb.summary["Flash Memory (MB)"] = flash_mem
wandb.summary["Flash + ALiBi Memory (MB)"] = flash_alibi_mem

wandb.summary["Standard ROUGE-1"] = standard_metrics[0]['rouge1']
wandb.summary["Flash ROUGE-1"] = flash_metrics[0]['rouge1']
wandb.summary["Flash + ALiBi ROUGE-1"] = flash_alibi_metrics[0]['rouge1']

wandb.summary["Standard ROUGE-2"] = standard_metrics[0]['rouge2']
wandb.summary["Flash ROUGE-2"] = flash_metrics[0]['rouge2']
wandb.summary["Flash + ALiBi ROUGE-2"] = flash_alibi_metrics[0]['rouge2']

wandb.summary["Standard ROUGE-L"] = standard_metrics[0]['rougeL']
wandb.summary["Flash ROUGE-L"] = flash_metrics[0]['rougeL']
wandb.summary["Flash + ALiBi ROUGE-L"] = flash_alibi_metrics[0]['rougeL']

wandb.summary["Standard BLEU"] = standard_metrics[1]
wandb.summary["Flash BLEU"] = flash_metrics[1]
wandb.summary["Flash + ALiBi BLEU"] = flash_alibi_metrics[1]

wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Flash + ALiBi BLEU,0.0327
Flash + ALiBi Inference Time (ms),30.94
Flash + ALiBi Memory (MB),3747.49
Flash + ALiBi ROUGE-1,0.1644
Flash + ALiBi ROUGE-2,0.0887
Flash + ALiBi ROUGE-L,0.1202
Flash BLEU,0.0296
Flash Inference Time (ms),17.21
Flash Memory (MB),3747.49
Flash ROUGE-1,0.1618
Flash ROUGE-2,0.0866


In [ ]:
# Fine-tune Flash attention model with ALiBi, since GPT-2's pre-trained weights are not adjusted to ALiBi scores
# Resize token embeddings if tokenizer vocabulary is changed
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})
    model_flash_with_alibi.resize_token_embeddings(len(tokenizer))

# Training Arguments
training_args = TrainingArguments(
    output_dir="./flash_alibi_summarization",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-5,
    warmup_steps=500,
    weight_decay=0.01,
    save_steps=1000,
    evaluation_strategy="steps",
    eval_steps=1000,
    logging_dir="./logs",
    logging_steps=500,
    save_total_limit=2,
    gradient_accumulation_steps=4,
    fp16=True,  # Enable mixed precision for faster training if supported
)

# Define Trainer
trainer = Trainer(
    model=model_flash_with_alibi,
    args=training_args,
    train_dataset=tokenized_train_dataset,  # Ensure these are tokenized datasets
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-53-446cd4a95195>:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=186, training_loss=2.435393302671371, metrics={'train_runtime': 54.09, 'train_samples_per_second': 55.463, 'train_steps_per_second': 3.439, 'total_flos': 777605087232000.0, 'train_loss': 2.435393302671371, 'epoch': 2.976})

In [ ]:
# Benchmarking
print("Running Benchmarks...")
standard_time, standard_mem = benchmark_model(model_standard, tokenizer, input_texts)
flash_time, flash_mem = benchmark_model(model_flash, tokenizer, input_texts)
flash_alibi_time, flash_alibi_mem = benchmark_model(model_flash_with_alibi, tokenizer, input_texts)

# Generate Texts
standard_outputs = generate_text(model_standard, tokenizer, input_texts)
flash_outputs = generate_text(model_flash, tokenizer, input_texts)
flash_alibi_outputs = generate_text(model_flash_with_alibi, tokenizer, input_texts)

# Compute Metrics
standard_metrics = compute_metrics(standard_outputs, reference_summaries)
flash_metrics = compute_metrics(flash_outputs, reference_summaries)
flash_alibi_metrics = compute_metrics(flash_alibi_outputs, reference_summaries)

# Organize Results
results_efficiency = [
    ["Model", "Inference Time (ms)", "Memory (MB)"],
    ["Standard GPT-2", f"{standard_time:.2f}", f"{standard_mem:.2f}"],
    ["Flash GPT-2", f"{flash_time:.2f}", f"{flash_mem:.2f}"],
    ["Flash + ALiBi GPT-2", f"{flash_alibi_time:.2f}", f"{flash_alibi_mem:.2f}"]
]

results_quality = [
    ["Model", "ROUGE-1", "ROUGE-2", "ROUGE-L", "BLEU"],
    ["Standard GPT-2", f"{standard_metrics[0]['rouge1']:.4f}", f"{standard_metrics[0]['rouge2']:.4f}", f"{standard_metrics[0]['rougeL']:.4f}", f"{standard_metrics[1]:.4f}"],
    ["Flash GPT-2", f"{flash_metrics[0]['rouge1']:.4f}", f"{flash_metrics[0]['rouge2']:.4f}", f"{flash_metrics[0]['rougeL']:.4f}", f"{flash_metrics[1]:.4f}"],
    ["Flash + ALiBi GPT-2", f"{flash_alibi_metrics[0]['rouge1']:.4f}", f"{flash_alibi_metrics[0]['rouge2']:.4f}", f"{flash_alibi_metrics[0]['rougeL']:.4f}", f"{flash_alibi_metrics[1]:.4f}"]
]

print("\n===== Efficiency Comparison =====\n")
print(tabulate(results_efficiency, headers="firstrow", tablefmt="pretty"))

print("\n===== Quality Comparison =====\n")
print(tabulate(results_quality, headers="firstrow", tablefmt="pretty"))

Running Benchmarks...


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Settin


===== Efficiency Comparison =====

+---------------------+---------------------+-------------+
|        Model        | Inference Time (ms) | Memory (MB) |
+---------------------+---------------------+-------------+
|   Standard GPT-2    |        72.41        |   4237.02   |
|     Flash GPT-2     |        15.46        |   3577.86   |
| Flash + ALiBi GPT-2 |        25.76        |   3578.98   |
+---------------------+---------------------+-------------+

===== Quality Comparison =====

+---------------------+---------+---------+---------+--------+
|        Model        | ROUGE-1 | ROUGE-2 | ROUGE-L |  BLEU  |
+---------------------+---------+---------+---------+--------+
|   Standard GPT-2    | 0.0870  | 0.0315  | 0.0578  | 0.0084 |
|     Flash GPT-2     | 0.0775  | 0.0282  | 0.0524  | 0.0075 |
| Flash + ALiBi GPT-2 | 0.0922  | 0.0336  | 0.0623  | 0.0089 |
+---------------------+---------+---------+---------+--------+


In [ ]:
# Initialize wandb
wandb.init(project="intro2llm", name="benchmark_results_finetuned")

# Log efficiency results
efficiency_table = wandb.Table(columns=["Model", "Inference Time (ms)", "Memory (MB)"])
efficiency_table.add_data("Standard GPT-2", standard_time, standard_mem)
efficiency_table.add_data("Flash GPT-2", flash_time, flash_mem)
efficiency_table.add_data("Flash + ALiBi GPT-2", flash_alibi_time, flash_alibi_mem)
wandb.log({"Efficiency Metrics": efficiency_table})

# Log quality results
quality_table = wandb.Table(columns=["Model", "ROUGE-1", "ROUGE-2", "ROUGE-L", "BLEU"])
quality_table.add_data("Standard GPT-2",
                       standard_metrics[0]['rouge1'],
                       standard_metrics[0]['rouge2'],
                       standard_metrics[0]['rougeL'],
                       standard_metrics[1])
quality_table.add_data("Flash GPT-2",
                       flash_metrics[0]['rouge1'],
                       flash_metrics[0]['rouge2'],
                       flash_metrics[0]['rougeL'],
                       flash_metrics[1])
quality_table.add_data("Flash + ALiBi GPT-2",
                       flash_alibi_metrics[0]['rouge1'],
                       flash_alibi_metrics[0]['rouge2'],
                       flash_alibi_metrics[0]['rougeL'],
                       flash_alibi_metrics[1])
wandb.log({"Quality Metrics": quality_table})

# Log as summary
wandb.summary["Standard Inference Time (ms)"] = standard_time
wandb.summary["Flash Inference Time (ms)"] = flash_time
wandb.summary["Flash + ALiBi Inference Time (ms)"] = flash_alibi_time

wandb.summary["Standard Memory (MB)"] = standard_mem
wandb.summary["Flash Memory (MB)"] = flash_mem
wandb.summary["Flash + ALiBi Memory (MB)"] = flash_alibi_mem

wandb.summary["Standard ROUGE-1"] = standard_metrics[0]['rouge1']
wandb.summary["Flash ROUGE-1"] = flash_metrics[0]['rouge1']
wandb.summary["Flash + ALiBi ROUGE-1"] = flash_alibi_metrics[0]['rouge1']

wandb.summary["Standard ROUGE-2"] = standard_metrics[0]['rouge2']
wandb.summary["Flash ROUGE-2"] = flash_metrics[0]['rouge2']
wandb.summary["Flash + ALiBi ROUGE-2"] = flash_alibi_metrics[0]['rouge2']

wandb.summary["Standard ROUGE-L"] = standard_metrics[0]['rougeL']
wandb.summary["Flash ROUGE-L"] = flash_metrics[0]['rougeL']
wandb.summary["Flash + ALiBi ROUGE-L"] = flash_alibi_metrics[0]['rougeL']

wandb.summary["Standard BLEU"] = standard_metrics[1]
wandb.summary["Flash BLEU"] = flash_metrics[1]
wandb.summary["Flash + ALiBi BLEU"] = flash_alibi_metrics[1]

wandb.finish()

wandb: Currently logged in as: rishabhsri66 (hpmlcolumbia). Use `wandb login --relogin` to force relogin


Flash + ALiBi BLEU,0.0089
Flash + ALiBi Inference Time (ms),25.76
Flash + ALiBi Memory (MB),3578.98
Flash + ALiBi ROUGE-1,0.0922
Flash + ALiBi ROUGE-2,0.0336
Flash + ALiBi ROUGE-L,0.0623
Flash BLEU,0.0075
Flash Inference Time (ms),15.46
Flash Memory (MB),3577.86
Flash ROUGE-1,0.0775
Flash ROUGE-2,0.0282
